# Iterative Deepening A* (IDA*)


The need to store in memory the A* search tree (=CLOSED and OPEN 'lists') may become a bottleneck for some problems. One of such problems is the well-known n-puzzle (gem puzzle) game (especially when n>3). One of the ways to mitigate this problem is to use iterative deepening technique in combination with A\*. This is known as IDA\* algorithm (Korf, 1985).

In this lab your task is to implement IDA* algorithm for n-puzzle game and compare against A* (with the main indicators being runtime (the number of steps performed by the algorithm) and memory consumption).

Good luck!

(Korf, 1985) Korf, R.E., 1985. Depth-first iterative-deepening: An optimal admissible tree search. Artificial intelligence, 27(1), pp.97-109. [[PDF](https://academiccommons.columbia.edu/doi/10.7916/D8BK1M9V/download)]

In [ ]:
import copy
import matplotlib.pyplot as plt
import math
from random import shuffle
from heapq import heappop, heappush


%matplotlib inline

## Representing the board state and the search state for the Gem Puzzle

In [ ]:
class GemPuzzleState:
    '''
    GemPuzzleState class represents state of the Gem Puzzle board
    
    - size: width of game field 
    
    - tile_list: tile positions represented as a list of integers. This list is expected 
                 to contain values from 1 to (size x size). Each integer value corresponds 
                 to a tile and the position in the list (index) corresponds to the position 
                 of the tile on the game field. Tile with the value (size x size) is assumed 
                 to represent blank position.
    
    - blank_pos: position of empty tile in tile_list. Explicitly storing the position of
                 a blank helps to generate successors faster.
    '''

    def __init__(self, tile_list, blank_pos = None):
        '''
        Constructor. Sets tile positions + some basic checks.
        '''
        
        self._tile_list = tile_list
        self._size = int(len(tile_list) ** 0.5)
        blank_value = self._size ** 2
        
        if (blank_value != len(tile_list)):
            raise Exception("The tile list must contain the number of \
                    elements which is equal to the square of an integer!")
        
        # Memorizing the position of a blank tile
        # Technically, there is no need to do so, but it makes to get the successors a bit faster
        if (blank_pos is not None):
            if (self._tile_list[blank_pos] != blank_value):
                raise Exception("Incorrect position of blank tile")
            self._blank_pos = blank_pos
        else:
            self._blank_pos = -1 
            for i, tile_value in enumerate(tile_list):
                if (tile_value == blank_value):
                    self._blank_pos = i

            if (self._blank_pos == -1):
                raise Exception("State should contains max value as position to blank tile")     
        
        # You may wish to add extra functionlality that might help in comparing board states
        self._state_hash = hash(str(self._tile_list))
                            
    
    def __eq__(self, other):
        '''
        Comparing one state with the other state. This is needed e.g. 
        to test whether we reached the goal state.
        '''
        return (self._state_hash == other._state_hash)
    

    def __hash__(self):
        '''
        You may wish to add extra functionlality that might help in comparing board states
        '''
        return self._state_hash


    def __str__(self):
        '''
        Converts game state to formatted string
        '''
        res = []
        char_tile_list = list(map(str, self._tile_list))
        char_tile_list[self.blank_pos] = '_'
        
        for row_start in range(0, len(char_tile_list), self._size):
            res.append(char_tile_list[row_start : row_start + self._size])
        
        return '\n'.join([''.join(['{:2}'.format(item) for item in row]) for row in res]) + "\n"
    
    
    @property
    def tile_list(self):
        return self._tile_list
    
    
    @property
    def blank_pos(self):
        return self._blank_pos
    
    
    @property
    def size(self):
        return self._size

## Search Node

A wrapper for the GemPuzzleState that incorporates the data needed for the search algorithms (g-values, h-values, etc.).

In [ ]:
class Node:
    '''
    Node class represents a search node

    - puzzle_state: the state of the game corresponding to the new node
    - g: g-value of the node
    - h: h-value of the node // always 0 for Dijkstra
    - F: f-value of the node // always equal to g-value for Dijkstra
    - parent: pointer to the parent-node 

    '''
    

    def __init__(self, puzzle_state, g = 0, h = 0, f = None, parent = None):
        self.puzzle_state = puzzle_state
        self.g = g
        self.h = h
        if f is None:
            self.f = self.g + h
        else:
            self.f = f        
        self.parent = parent

        
    
    def __eq__(self, other):
        '''
        Estimating where the two search nodes are the same,
        which is needed to detect dublicates in the search tree.
        '''
        return self.puzzle_state == other.puzzle_state
    
    
    def __hash__(self):
        '''
        To implement CLOSED for A* as set of nodes we need Node to be hashable.
        '''
        return hash(self.puzzle_state)
    
    
    def __str__(self):
        '''
        Converts corrsponding game state to formatted string
        '''
        return str(self.puzzle_state)


    def __lt__(self, other): 
        '''
        Comparing the keys (i.e. the f-values) of two nodes,
        which is needed to sort/extract the best element from OPEN (for A*).
        '''
        return self.f < other.f

## Succesors

In [ ]:
def get_successors(state):
    '''
    Getting the succesors for the GemPuzzleState/SearchNode (both variants are OK, it's up to you).
    '''
    delta = [[0, 1], [1, 0], [0, -1], [-1, 0]]
    successors = []
    for d in delta:
        row = state.blank_pos // state.size # identifying the row in which blank is located
        col = state.blank_pos % state.size # identifying the column in which blank is located
        row += d[0] # computing new row for blank (corresponding to a particular move encoded via 'd')
        col += d[1] # computing new column for blank (corresponding to a particular move encoded via 'd')
        
        # if the new position of a blank is valid (i.e. it is still within the field) then
        # a corresponding sucessor should be added to the succesors' list
        if (0 <= row < state.size) and (0 <= col < state.size):
            new_tile_list = copy.copy(state.tile_list)
            new_blank_pos = row * state.size + col
            new_tile_list[state.blank_pos] = new_tile_list[new_blank_pos] # moving tile
            new_tile_list[new_blank_pos] = state.size ** 2 # setting blank            
            new_state = GemPuzzleState(new_tile_list, new_blank_pos)
            successors.append(new_state)
        
    return successors

## Heuristics Search
### Implementing the Search Tree (i.e. OPEN and CLOSED) for A*

You can use the implementation of the search tree from the previous labs. If it was implemented correctly, no modifications are needed.

In [ ]:
class SearchTreePQS: #SearchTree which uses PriorityQueue for OPEN and set for CLOSED
    
    def __init__(self):
        pass
                                      
    def __len__(self):
        pass
                
    def open_is_empty(self):
        '''
        open_is_empty should inform whether the OPEN is exhausted or not.
        In the former case the search main loop should be interrupted.
        '''
        pass
    
 
    def add_to_open(self, item):
        '''
        Adding a (previously not expanded) node to the search-tree (i.e. to OPEN).
        It's either a totally new node (the one we never encountered before)
        or it can be a dublicate of the node that currently resides in OPEN.
        It's up to us how to handle dublicates in OPEN. We can either 
        detect dublicates upon adding (i.e. inside this method) or detect them
        lazily, when we are extracting a node for further expansion.
        Not detecting dublicates at all may work in certain setups but let's not
        consider this option.
        '''   
        pass

    def get_best_node_from_open(self):
        '''
        Extracting the best node (i.e. the one with the minimal key) from OPEN.
        This node will be expanded further on in the main loop of the search.
        ''' 

        pass

    def add_to_closed(self, item):
        pass

    def was_expanded(self, item):
        pass

    @property
    def opened(self):
        pass
    
    @property
    def expanded(self):
        pass

    @property
    def number_of_open_dublicates(self):
        pass
    

### Heuristics
You need to implement 2 most common *admissible* heuristic functions for the n-puzzle.
- Hamming distance (the number of the misplaced tiles on the board).
- Manhattan distance (Google it).

In [ ]:
def hamming_distance(state_1, state_2):
    '''
    TODO
    '''
    h = 0

    return h

In [ ]:
def manhattan_distance(state_1, state_2):
    '''
    TODO
    '''
    h = 0

    return h


### A* Algorithm

Adapt the A* algorithm for the gem puzzle

The output of the algorithm should be:
- path found flag (boolean)
- the last (goal) node (needed further to reconstruct a path)
- number of expansions (= steps) an algorithm have made until reaching the solution
- number of created (and stored in the memory) nodes(= memory usage)

In [ ]:
def astar(start_state, goal_state, heuristic_function = manhattan_distance, search_tree = None):
    '''
    TODO
    '''
    
    ast = search_tree() # A* search tree
    start_node = Node(start_state, g=0, h=heuristic_function(start_state, goal_state))
    ast.add_to_open(start_node)
    
    steps = 0
    nodes_created = 0
        

    return False, None, steps, nodes_created 

### IDA* Algorithm

Implement IDA* algorithm. 

No other nodes except the ones forming the current path, i.e. the branch in the search tree, have to be memorized.

Checking for dublicates checks whether the generated node is already present in the current path.

The output of the algorithm should be analogous to A*.

In [ ]:
def search(curr_node, goal_state, bound, path, steps, heuristic_function):
    steps += 1
    f = curr_node.f
    
    if f > bound:
        return (False, f, None, path, steps)

    if curr_node.puzzle_state == goal_state:
        return (True, f, curr_node, path, steps)

    next_f = math.inf

    '''
    TODO
    '''
            
    return False, next_f, None, path, steps

def idastar(start_state, goal_state, heuristic_function = manhattan_distance):
    bound = heuristic_function(start_state, goal_state)
    start_node = Node(start_state, g=0, h=heuristic_function(start_state, goal_state))
    path = set()
    steps = 0
    
    '''
    TODO
    '''
    
    
    res_node = start_node
    res_path = path
    return False, res_node, steps, len(res_path)


## Validating the Results
The following code randomly generates a bunch of solvable test instances (8-puzzle) and puts them into `data/tasks_gem.txt`.

In [ ]:
def is_solvable(tile_list):
    inversions = 0
    puzzle_exept_empty = [(i, v) for i, v in enumerate(tile_list) if v != len(tile_list)]

    for i, tile in puzzle_exept_empty:
        j = i + 1
        while j < len(tile_list):
            if tile_list[j] < tile:
                inversions += 1
            j += 1
    
    size = int(math.sqrt(len(tile_list)))
    
    if size % 2 != 0 and inversions % 2 == 0:
        return True

    if size % 2 == 0:
        empty_row = size - puzzle.index(len(tile_list)) // size
        return (empty_row % 2 != 0) == (inversions % 2 == 0)

    return False

In [ ]:
def generate_tasks(task_file, number, size):
    f = open(task_file, "w")
    for i in range(number):
        tile_list = [i + 1 for i in range(size ** 2)]
        shuffle(tile_list)
        goal_tile_list = list(range(1, len(tile_list) + 1))
        start_state = GemPuzzleState(tile_list)
        goal_state = GemPuzzleState(goal_tile_list)
        dist = manhattan_distance(start_state, goal_state)
        
        while (not is_solvable(tile_list)) or (dist > 12):
            shuffle(tile_list)
            goal_tile_list = list(range(1, len(tile_list) + 1))
            start_state = GemPuzzleState(tile_list)
            goal_state = GemPuzzleState(goal_tile_list)
            dist = manhattan_distance(start_state, goal_state)

        f.write(" ".join(map(str, tile_list)) + "\n")

        print(*tile_list, "Manhattan distance", dist)


In [ ]:
# Use it only once
generate_tasks('data/tasks_gem.txt', 30, 3)

The following procedure takes a search algorithm and a heuristic function (e.g. IDA* + Manhattan distance) as parameters, runs tests and stores the results for further analysis.

In [ ]:
def massive_test(SearchFunction, *args):
    tasks_file = open('data/tasks_gem.txt')
    all_tasks_results = dict()
    all_tasks_results["len"] = []
    all_tasks_results["nc"] = []
    all_tasks_results["st"] = []
    count = 0
    for l in tasks_file:
        count += 1
        
        if(len(l) == 0):
            continue
            
        start_tile_list = list(map(int, l.split()))
        goal_tile_list = list(range(1, len(start_tile_list) + 1))
        start_state = GemPuzzleState(start_tile_list)
        goal_state = GemPuzzleState(goal_tile_list)
        try:
            result = SearchFunction(start_state, goal_state, *args)

            if(result[0]):
                print(result[1].g, result[2], result[3])
                all_tasks_results["len"].append(result[1].g)
                all_tasks_results["st"].append(result[2])
                all_tasks_results["nc"].append(result[3])
            else:
                print(count, "Path not found!")

        except Exception as e:
            print(count, "Execution error")
            print(e)

    return all_tasks_results

## Experiment
Run `A*` and `IDA*` with different heuristics on the generated instances and collect the results for further analysis.

In [ ]:
%time astar_manh_stat = massive_test(astar, manhattan_distance, SearchTreePQS)

In [ ]:
%time astar_hamm_stat = massive_test(astar, hamming_distance, SearchTreePQS)

In [ ]:
%time idastar_manh_stat = massive_test(idastar, manhattan_distance)

In [ ]:
%time idastar_hamm_stat = massive_test(idastar, hamming_distance)

## Analisys 
Let's create very simple plots to analyze the performance difference of A* vs IDA*.

In [ ]:
len_eq_count = 0
for i in range(len(astar_manh_stat['len'])):
    if astar_manh_stat['len'][i] == astar_hamm_stat['len'][i] == \
               idastar_manh_stat['len'][i] == idastar_hamm_stat['len'][i]:
        len_eq_count += 1
               
print("The path lengths are the same in", len_eq_count/len(astar_manh_stat['len']) * 100, '% of cases')

astar_manh_mem = sum(astar_manh_stat['nc'])/len(astar_manh_stat['nc'])
astar_hamm_mem = sum(astar_hamm_stat['nc'])/len(astar_hamm_stat['nc'])
idastar_manh_mem = sum(idastar_manh_stat['nc'])/len(idastar_manh_stat['nc'])
idastar_hamm_mem = sum(idastar_hamm_stat['nc'])/len(idastar_hamm_stat['nc'])

astar_manh_steps = sum(astar_manh_stat['st'])/len(astar_manh_stat['st'])
astar_hamm_steps = sum(astar_hamm_stat['st'])/len(astar_hamm_stat['st'])
idastar_manh_steps = sum(idastar_manh_stat['st'])/len(idastar_manh_stat['st'])
idastar_hamm_steps = sum(idastar_hamm_stat['st'])/len(idastar_hamm_stat['st'])

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.set_title('Average number of nodes stored in memory (= memory usage)')
alg = ['A* (Manh)', 'A* (Hamm)', 'IDA* (Manh)', 'IDA* (Hamm)']
mem = [astar_manh_mem, astar_hamm_mem, idastar_manh_mem, idastar_hamm_mem]
ax.bar(alg, mem)
plt.show()

astar_manh_mem, astar_hamm_mem, idastar_manh_mem, idastar_hamm_mem

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.set_title('Average number of steps')
alg = ['A* (Manh)', 'A* (Hamm)', 'IDA* (Manh)', 'IDA* (Hamm)']
sts = [astar_manh_steps, astar_hamm_steps, idastar_manh_steps, idastar_hamm_steps]
ax.bar(alg, sts)
plt.show()

astar_manh_steps, astar_hamm_steps, idastar_manh_steps, idastar_hamm_steps

## Real memory consumption

Finally, lets measure the real time/memory consumption on a certain 'not-so-easy' instance of 15-puzzle.

In [ ]:
# to measure the memory consumtion in absolute values you can use the 'memory_profiler' 
# package (you may need installation)

%reload_ext memory_profiler

In [ ]:
start_tile_list_str = "1 2 10 8 12 14 6 4 15 13 5 3 9 7 11 16" # You can try a harder instance if you like
start_tile_list = list(map(int, start_tile_list_str.split()))
goal_tile_list = list(range(1, len(start_tile_list) + 1))
start_state = GemPuzzleState(start_tile_list)
goal_state = GemPuzzleState(goal_tile_list)

In [ ]:
%%time
%memit idastar(start_state, goal_state, manhattan_distance)

In [ ]:
%%time
%memit astar(start_state, goal_state, manhattan_distance, SearchTreePQS)